In [1]:
# coding=utf-8
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)
# driver = webdriver.Firefox()

first = True
def add_info(url):
	global first
	driver.get(url)
	wait = WebDriverWait(driver, 20)
	if "gehalt.de" in url:
		wait.until(lambda driver: driver.current_url != url)
	link = driver.current_url
	if "stepstone.de" not in link:
		return(link, 
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",
				"Nicht stepstone",)
	# print(link)
	if first:
		wait.until(lambda driver: len(driver.find_elements(By.CSS_SELECTOR, "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button")) > 0)
		driver.find_element(By.CSS_SELECTOR, "#ccmgt_explicit_preferences.privacy-prompt-button.secondary-button.options-button").click()
		driver.find_element(By.CSS_SELECTOR, ".secondary-button.ccmgt_reject_button").click()
		first = False

	if len(driver.find_elements(By.CLASS_NAME, "listing-content-provider-hw0nf9")) > 0:
		driver.find_element(By.CLASS_NAME, "listing-content-provider-hw0nf9").click()
	
	response = driver.page_source
	soup_object = BeautifulSoup(response)
	if len(soup_object.find_all(attrs={"data-genesis-element":"ALERT_CONTENT"})) > 0:
		return(link, 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar", 
				"Stellenanzeige nicht mehr verfügbar")
	# Teilzeit und Remote
	if len(soup_object.find_all(class_="at-listing__list-icons_work-type")) > 0:
		work_type = soup_object.find(class_="at-listing__list-icons_work-type").find(attrs={"data-genesis-element":"TEXT"})
		# # print(soup_object)
		# print(work_type)
		teilzeit_remote = work_type.text
	else:
		teilzeit_remote = "-"

	base = soup_object.find_all("div", class_="sc-EHOje jWEhHL")

	texts = ["", "", "", "", ""]

	for i in range(len(base)):
		area = base[i].find_all(["p", "li"])
		for element in area:
			texts[i] += element.text + "\n"
	
	if len(base) > 5:
		print(len(base))

	return link, teilzeit_remote, texts[0], texts[1], texts[2], texts[3], texts[4]

# jobs = pd.read_pickle("jobs.pkl")

# jobs1 = jobs[jobs["Teilzeit_Remote"].isna() == True]
# indexes = jobs1.index
# for i in indexes:
# 	jobs.loc[i, ["Link", "Teilzeit_Remote", "Introduction", "Description", "Profile", "We_offer", "Contact"]] = add_info(jobs1.loc[i, "Link"])
# 	jobs.to_pickle("jobs.pkl")
# driver.quit()
# jobs


In [2]:
jobs = pd.read_pickle("../data/jobs.pkl")
jobs1 = jobs[jobs["Teilzeit_Remote"] == "-"]
jobs1


,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact
270,Sales Data Analyst (m/w/d),Borne Firmengruppe,Trierweiler,,,57542587,https://www.stepstone.de/stellenangebote--sale...,2023-09-04,-,\nHinter Borne stehen mehr als 65 Jahre gelebt...,\nSie erstellen zahlenbasierte Analysen der Ve...,"\nStudium der Wirtschaftswissenschaften, Wirsc...",\ngroßen Freiraum Verantwortung zu übernehmen ...,\n Markus Mittal\n\n\n 0651/8106370\n\n
503,Manager Brand Data Architecture (m/f/d),adidas AG,Herzogenaurach,,,57459419,https://www.stepstone.de/stellenangebote--mana...,2023-09-04,-,\nThis role will oversee allocated product dat...,\nLeads and supports workstreams in product da...,\nExperience with developing enterprise data m...,\nAT ADIDAS WE HAVE A WINNING CULTURE. BUT TO ...,
582,Employees in purchasing and master data mainte...,WENZEL Group GmbH Co & KG,Wiesthal,,,57405442,https://www.stepstone.de/stellenangebote--empl...,2023-09-04,-,\nThe WENZEL Group is one of the leading suppl...,\nCarry out inquiry process with suppliers\n\n...,\nA completed commercial or technical educatio...,\nBy giving you a great deal of creative freed...,\n Are you curious? Then we look forward to re...
811,Junior-Anwendungsentwickler (m/w/d),FENECON GmbH & Co. KG,Deggendorf,,,57542116,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,-,\nZur Unterstützung unseres Teams suchen wir a...,\nAnwendungsentwicklung im Bereich Energiemana...,\nErfolgreich abgeschlossenes Studium im Berei...,\nAttraktives Vergütungsmodell\n\n\nJährlicher...,\n Du hast Fragen? Ich bin gerne für dich da.\...
1093,Junior Sales Agent (m/w/d),Baufinanzierungspool24 GmbH & Co KG,Karlsruhe,,,57419071,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,-,Herzlich Willkommen bei Baufinanzierungspool24...,\nAls Teil unseres Vertriebsteams unterstützt ...,"\nSpaß am Telefonieren, sowie ausgeprägte Komm...",\nUnbefristete Festanstellung in Vollzeit in e...,\n Bewirb Dich bitte direkt online\n \n\n Dein...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7473,Team Lead (m/w/d) Data Relationship Management...,Vaillant Group Business Services GmbH,Remscheid,7903,10000,57698827,https://www.stepstone.de/stellenangebote--team...,2023-09-19,-,,,,,
7484,(Junior-) Referent PR und Marketing (m/w/d),Grünecker Patent- und Rechtsanwälte PartG mbB,München,3468,4597,57670076,https://www.stepstone.de/stellenangebote--juni...,2023-09-19,-,,,,,
7490,Techniker* / Junior Ingenieur* Automotive Engi...,EDAG Engineering GmbH,Wolfsburg,,,57660201,https://www.stepstone.de/stellenangebote--tech...,2023-09-19,-,,,,,
7491,(Junior) Projektingenieur / Industrial Enginee...,Einhell Germany AG,"Landau an der Isar, Homeoffice möglich",,,57724798,https://www.stepstone.de/stellenangebote--Juni...,2023-09-19,-,,,,,


In [3]:


indexes = jobs1.index
for i in indexes:
	jobs.loc[i, ["Link", "Teilzeit_Remote", "Introduction", "Description", "Profile", "We_offer", "Contact"]] = add_info(jobs1.loc[i, "Link"])
	# jobs.to_pickle("../data/jobs.pkl")
driver.quit()

In [9]:
jobs1 = jobs[jobs["Teilzeit_Remote"] == ""]
jobs1

,Titel,Unternehmen,Ort,Gehalt_min,Gehalt_max,JobID,Link,Datum,Teilzeit_Remote,Introduction,Description,Profile,We_offer,Contact
270,Sales Data Analyst (m/w/d),Borne Firmengruppe,Trierweiler,,,57542587,https://www.stepstone.de/stellenangebote--sale...,2023-09-04,,\nHinter Borne stehen mehr als 65 Jahre gelebt...,\nSie erstellen zahlenbasierte Analysen der Ve...,"\nStudium der Wirtschaftswissenschaften, Wirsc...",\ngroßen Freiraum Verantwortung zu übernehmen ...,
503,Manager Brand Data Architecture (m/f/d),adidas AG,Herzogenaurach,,,57459419,https://www.stepstone.de/stellenangebote--mana...,2023-09-04,,\nThis role will oversee allocated product dat...,\nLeads and supports workstreams in product da...,\nExperience with developing enterprise data m...,\nAT ADIDAS WE HAVE A WINNING CULTURE. BUT TO ...,
582,Employees in purchasing and master data mainte...,WENZEL Group GmbH Co & KG,Wiesthal,,,57405442,https://www.stepstone.de/stellenangebote--empl...,2023-09-04,,\nThe WENZEL Group is one of the leading suppl...,\nCarry out inquiry process with suppliers\n\n...,\nA completed commercial or technical educatio...,\nBy giving you a great deal of creative freed...,
811,Junior-Anwendungsentwickler (m/w/d),FENECON GmbH & Co. KG,Deggendorf,,,57542116,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,,\nZur Unterstützung unseres Teams suchen wir a...,\nAnwendungsentwicklung im Bereich Energiemana...,\nErfolgreich abgeschlossenes Studium im Berei...,\nAttraktives Vergütungsmodell\n\n\nJährlicher...,
1093,Junior Sales Agent (m/w/d),Baufinanzierungspool24 GmbH & Co KG,Karlsruhe,,,57419071,https://www.stepstone.de/stellenangebote--juni...,2023-09-04,,Herzlich Willkommen bei Baufinanzierungspool24...,\nAls Teil unseres Vertriebsteams unterstützt ...,"\nSpaß am Telefonieren, sowie ausgeprägte Komm...",\nUnbefristete Festanstellung in Vollzeit in e...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6709,Data Engineer,Klarna GmbH,"BERLIN, GERMANY",,,57711910,https://www.stepstone.de/stellenangebote--data...,2023-09-15,,\nOn our path to becoming the world’s favorite...,\nAs a data engineer you will work in a team t...,\nRelevant education with a degree in engineer...,\nDiversity & Community: With our diversity of...,
6856,Senior Data Scientist - Risk Modeling,Klarna GmbH,"BERLIN, GERMANY",,,57726671,https://www.stepstone.de/stellenangebote--seni...,2023-09-19,,\nOn our path to becoming the world’s favorite...,\nLending is one of Klarna's main business act...,\nSeveral years of experience working in eithe...,\nDiversity & Community: With our diversity of...,
6897,Data Engineer,Klarna GmbH,"BERLIN, GERMANY",,,57726660,https://www.stepstone.de/stellenangebote--data...,2023-09-19,,\nOn our path to becoming the world’s favorite...,\nAs a data engineer you will work in a team t...,\nRelevant education with a degree in engineer...,\nDiversity & Community: With our diversity of...,
7003,Junior Accountant,Klarna GmbH,"BERLIN, GERMANY",,,57726659,https://www.stepstone.de/stellenangebote--juni...,2023-09-19,,\nOn our path to becoming the world’s favorite...,• Assisting with bank reconciliations and oth...,• Bachelor's degree in Accounting or related ...,\nDiversity & Community: With our diversity of...,


In [10]:
jobs.loc[jobs1.index, "Teilzeit_Remote"] = "-"


In [11]:
jobs.loc[6709]

Titel                                                  Data Engineer
Unternehmen                                              Klarna GmbH
Ort                                                  BERLIN, GERMANY
Gehalt_min                                                          
Gehalt_max                                                          
JobID                                                       57711910
Link               https://www.stepstone.de/stellenangebote--data...
Datum                                            2023-09-15 00:00:00
Teilzeit_Remote                                                    -
Introduction       \nOn our path to becoming the world’s favorite...
Description        \nAs a data engineer you will work in a team t...
Profile            \nRelevant education with a degree in engineer...
We_offer           \nDiversity & Community: With our diversity of...
Contact                                                             
Name: 6709, dtype: object

In [12]:
jobs.to_pickle("../data/jobs.pkl")